In [1]:
# definitive 

In [93]:
# =============================================================================
# CELL 1: IMPORTS & CONFIGURATION
# =============================================================================

import pandas as pd
import statsmodels.formula.api as smf
import os
import numpy as np
import re
import warnings
import matplotlib.pyplot as plt # Import for plotting
import statsmodels.tools.sm_exceptions # Import for warning handling

# Suppress common warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=statsmodels.tools.sm_exceptions.ValueWarning)

In [94]:
# =============================================================================
# CELL 2: HELPER FUNCTIONS (PLOTTING & PRINTING)
# =============================================================================

def print_did_event_study_coeffs(results, title, variable="MONTH", ddd_term=None, policy_month=7):
    """
    Finds, prints, and returns a DataFrame of event study coefficients
    from a statsmodels results object.
    """
    print("\n" + "="*60 + f"\n>>> Event Study Numerical Results: {title} <<<" + "\n" + "="*60)
    
    if ddd_term:
        interaction_full = rf"C\(TreatState\)\[T\.1\]:C\({variable}[^\]]*\).*?\[T\.(\d+)\]:{ddd_term}$"
        print(f"Coefficients for: C(TreatState)*C({variable})*{ddd_term} interactions")
    else:
        interaction_full = rf"C\(TreatState\)\[T\.1\]:C\({variable}[^\]]*\).*?\[T\.(\d+)\]$"
        print(f"Coefficients for: C(TreatState)*C({variable}) interactions")
        
    print(f"Checking PTA: Pre-trend p-values (Months < {policy_month}) should be > 0.10")
    print("-" * 60)
    
    did_terms = []
    term_pattern = re.compile(interaction_full, re.IGNORECASE)
    
    ref_month = 2 # Defaulting to 2 (February)
    try:
        ref_match = re.search(r'C\(MONTH, Treatment\(reference=(\d+)\)\)', results.model.formula)
        if ref_match:
            ref_month = int(ref_match.group(1))
    except:
        pass # Keep default

    for var in results.params.index:
        match = term_pattern.search(var)
        if match:
            month = int(match.group(1))
            did_terms.append({
                'relative_month': month, 
                'coef': results.params[var],
                'pval': results.pvalues[var],
                'stderr': results.bse[var]
            })
            
    if not did_terms:
        print("No event study coefficients found.")
        print(f"Pattern searched for: {interaction_full}")
        print("-" * 60)
        return None
        
    est_df = pd.DataFrame(did_terms).sort_values('relative_month')
    
    if ref_month not in est_df['relative_month'].values:
        ref_month_df = pd.DataFrame([{'relative_month': ref_month, 'coef': 0, 'stderr': 0, 'pval': 1.0}])
        plot_df = pd.concat([est_df, ref_month_df]).sort_values('relative_month').reset_index(drop=True)
    else:
        plot_df = est_df.copy()
    
    for _, row in plot_df.iterrows():
        print(f"Month {int(row.relative_month):>2}:  Coef = {row.coef:>7.4f},  SE = {row.stderr:>6.4f},  P = {row.pval:.4f}")
    print("-" * 60)
    
    pre_trend_pvals = plot_df[plot_df['relative_month'] < policy_month]['pval']
    if pre_trend_pvals.empty:
        print(f"No pre-trends (< {policy_month}) to test.")
    elif (pre_trend_pvals[plot_df['relative_month'] != ref_month] > 0.10).all():
        print("SUCCESS: Parallel Trends Assumption appears satisfied.")
    else:
        print("WARNING: Parallel Trends Assumption may be VIOLATED.")
        
    return plot_df

def plot_did_event_study(plot_df, title):
    """
    Plots the event study coefficients and 95% confidence intervals.
    """
    if plot_df is None or plot_df.empty:
        print(f"Skipping plot {title}: No data.");
        return
        
    plt.figure(figsize=(10, 6))
    plt.axhline(0, color='black', linewidth=1, linestyle='-')
    plt.axvline(6.5, color='gray', linestyle='--', linewidth=1, label='Policy Change (July)') 
    
    plot_df['ci_low'] = plot_df['coef'] - 1.96 * plot_df['stderr']
    plot_df['ci_high'] = plot_df['coef'] + 1.96 * plot_df['stderr']
    
    plt.fill_between(plot_df['relative_month'], plot_df['ci_low'], plot_df['ci_high'], color='lightblue', alpha=0.4, label='95% CI')
    plt.plot(plot_df['relative_month'], plot_df['coef'], marker='o', linestyle='-', color='darkblue', label='DiD Coefficient')
    
    plt.title(title, fontsize=16)
    plt.xlabel("Calendar Month", fontsize=12) 
    plt.ylabel("DDD Coefficient (PP)", fontsize=12)
    plt.xticks(plot_df['relative_month'].unique()) 
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    
    safe_title = title.replace(' ','_').replace('(','').replace(')','').replace(',','').lower()
    plt.savefig(f"{safe_title}.png")
    plt.close()

In [95]:
# =============================================================================
# CELL 3: CORE ANALYSIS FUNCTION
# =============================================================================

def run_state_did_analysis(df, year, holzer_treat_fips, holzer_control_fips, monthly_covid, monthly_stringency, control_vars, policy_month=7):
    """
    Main analysis function.
    It will only add controls specified in 'control_vars'.
    """
    
    print("\n" + "="*80)
    print(f"--- RUNNING MAIN ANALYSIS (LowWage) FOR YEAR: {year} ---")
    print("="*80)

    # --- Step 3: Prepare Final STATE Panel ---
    if 'IND' not in df.columns:
        print("ERROR: 'IND' (Industry) not found in CPS data.")
        return
    
    df_year = df[df['YEAR'] == year].copy()
    if df_year.empty:
        print(f"ERROR: No data found for {year}.")
        return
    
    included_states = holzer_treat_fips.union(holzer_control_fips)
    
    LOW_WAGE_INDUSTRIES = list(range(8560, 8700)) + list(range(4670, 5800))
    df_year['IND'] = pd.to_numeric(df_year['IND'], errors='coerce')
    
    def define_low_wage(x):
        if x in LOW_WAGE_INDUSTRIES: return 1
        elif x > 0: return 0
        else: return np.nan 

    df_year['LowWage'] = df_year['IND'].apply(define_low_wage)
    
    df_year = df_year[df_year['STATEFIP'].isin(included_states)].copy()
    df_year['TreatState'] = df_year['STATEFIP'].apply(lambda x: 1 if x in holzer_treat_fips else 0)
    
    df_year.sort_values(['CPSIDP', 'YEAR', 'MONTH'], inplace=True)
    df_year['status_next_month'] = df_year.groupby('CPSIDP')['EMPSTAT'].shift(-1)
    unemployed_df = df_year[df_year['EMPSTAT'].between(20, 22)].copy() 
    unemployed_df['found_job'] = unemployed_df['status_next_month'].between(10, 12).astype(int) 
    
    unemployed_df['Post'] = ((unemployed_df['MONTH'] >= policy_month) & (unemployed_df['MONTH'] <= policy_month + 1)).astype(int)
    final_panel = unemployed_df[unemployed_df['MONTH'].between(2, 8)].copy()
    
    final_panel = final_panel.dropna(subset=['found_job', 'LNKFW1MWT', 'LowWage']) 
    final_panel = final_panel[final_panel['LNKFW1MWT'] > 0] 
    final_panel['LowWage'] = final_panel['LowWage'].astype(int) 
    
    print(f"Base {year} panel (Holzer States, Feb-Aug, valid industry): {len(final_panel)} obs")
    print(f"  Other Wage (n={len(final_panel[final_panel['LowWage']==0])}), Low Wage (n={len(final_panel[final_panel['LowWage']==1])})")

    # --- Step 4: Merge Controls ---
    monthly_covid_year = monthly_covid[monthly_covid['YEAR'] == year]
    final_panel_with_controls = pd.merge(final_panel, monthly_covid_year, on=['STATEFIP', 'YEAR', 'MONTH'], how='left')
    
    current_control_vars = []
    
    if year > 2019 and 'log_monthly_cases' in control_vars:
        current_control_vars.append('log_monthly_cases')
    else:
        final_panel_with_controls['log_monthly_cases'] = final_panel_with_controls['log_monthly_cases'].fillna(0)

    # This 'if' block will now be false and skipped, as monthly_stringency is empty
    if not monthly_stringency.empty and 'avg_stringency' in control_vars:
        monthly_stringency_year = monthly_stringency[monthly_stringency['YEAR'] == year]
        if not monthly_stringency_year.empty:
            final_panel_with_controls = pd.merge(final_panel_with_controls, monthly_stringency_year, on=['STATEFIP', 'YEAR', 'MONTH'], how='left')
            current_control_vars.append('avg_stringency')
        
    original_count = len(final_panel_with_controls)
    
    if current_control_vars:
        final_panel_with_controls.dropna(subset=current_control_vars, inplace=True)

    dropped_count = original_count - len(final_panel_with_controls)
    if dropped_count > 0:
        print(f"Warning: Dropped {dropped_count} observations missing control data.")
        
    print(f"Final {year} panel with controls: {len(final_panel_with_controls)} obs")
    if len(final_panel_with_controls) == 0:
        print(f"ERROR: Panel for {year} is empty after merging controls."); return

    # --- Step 5: Run State-Level DDD Model (LowWage) ---
    formula_parts = ["found_job ~ TreatState * Post * LowWage", "C(STATEFIP)", "C(MONTH)"]
    
    if 'log_monthly_cases' in current_control_vars:
        formula_parts.insert(1, "log_monthly_cases")
    # This will now be false
    if 'avg_stringency' in current_control_vars:
        formula_parts.insert(1, "avg_stringency")
        
    final_formula = " + ".join(formula_parts)
    
    model_ddd = smf.wls(formula=final_formula, data=final_panel_with_controls, weights=final_panel_with_controls['LNKFW1MWT'])
    results_ddd = model_ddd.fit(cov_type='cluster', cov_kwds={'groups': final_panel_with_controls['STATEFIP']})
    
    ddd_term = 'TreatState:Post:LowWage'
    if ddd_term in results_ddd.params:
        coef_ddd = results_ddd.params[ddd_term]
        pval_ddd = results_ddd.pvalues[ddd_term]
        
        print(f"\n--- DDD Results ({year}, All Ages, Holzer States, LowWage) ---")
        print(f"Key Coefficient ({ddd_term}): {coef_ddd:.4f}")
        print(f"P-value: {pval_ddd:.4f}\n")
        
        print("--- Coefficient Summary (Key Variables) ---")
        key_vars_to_print = [ddd_term, 'TreatState:Post', 'TreatState:LowWage', 'Post:LowWage'] + [v for v in current_control_vars if v in results_ddd.params.index]
        for var in key_vars_to_print:
            if var in results_ddd.params:
                print(f"{var:<25}: Coef = {results_ddd.params[var]:>8.4f}, P-val = {results_ddd.pvalues[var]:.4f}")
        
        print("\n--- Interpretation ---")
        if year == 2018:
            if pval_ddd < 0.10: print(f"PLACEBO TEST FAILED: Found a significant effect (p={pval_ddd:.3f}) in placebo year 2018.")
            else: print(f"PLACEBO TEST PASSED (2018): No significant effect found (p={pval_ddd:.3f}).")
        else:
            if pval_ddd < 0.10: print(f"Result: Found a statistically significant differential effect (p={pval_ddd:.3f}).")
            else: print(f"Result: No statistically significant differential effect found (p={pval_ddd:.3f}).")

    else: print(f"ERROR: '{ddd_term}' coefficient was not estimated.")

    # --- Step 5b: Run Separate DiD Models by Wage Group ---
    print("\n" + "="*80)
    print(f">>> Step 5b: Separate 2x2 DiD Models by Wage Group ({year}, LowWage) <<<")
    print("="*80)
    
    other_wage_panel = final_panel_with_controls[final_panel_with_controls['LowWage'] == 0].copy()
    low_wage_panel = final_panel_with_controls[final_panel_with_controls['LowWage'] == 1].copy()

    formula_parts_simple_did = ["found_job ~ TreatState * Post", "C(STATEFIP)", "C(MONTH)"]
    if 'log_monthly_cases' in current_control_vars:
        formula_parts_simple_did.insert(1, "log_monthly_cases")
    if 'avg_stringency' in current_control_vars:
        formula_parts_simple_did.insert(1, "avg_stringency")
    simple_did_formula = " + ".join(formula_parts_simple_did)
    
    model_did_other = smf.wls(formula=simple_did_formula, data=other_wage_panel, weights=other_wage_panel['LNKFW1MWT'])
    results_did_other = model_did_other.fit(cov_type='cluster', cov_kwds={'groups': other_wage_panel['STATEFIP']})
    
    coef_did_other = results_did_other.params.get('TreatState:Post', np.nan)
    pval_did_other = results_did_other.pvalues.get('TreatState:Post', np.nan)
    
    print(f"\n--- DiD Results for OTHER WAGE Group (LowWage=0) [{year}] ---")
    print(f"Key Coefficient (TreatState:Post): {coef_did_other:.4f}")
    print(f"P-value: {pval_did_other:.4f}")

    model_did_low = smf.wls(formula=simple_did_formula, data=low_wage_panel, weights=low_wage_panel['LNKFW1MWT'])
    results_did_low = model_did_low.fit(cov_type='cluster', cov_kwds={'groups': low_wage_panel['STATEFIP']})
    
    coef_did_low = results_did_low.params.get('TreatState:Post', np.nan)
    pval_did_low = results_did_low.pvalues.get('TreatState:Post', np.nan)
    
    print(f"\n--- DiD Results for LOW WAGE Group (LowWage=1) [{year}] ---")
    print(f"Key Coefficient (TreatState:Post): {coef_did_low:.4f}")
    print(f"P-value: {pval_did_low:.4f}")
    
    # --- Step 6: Run Event Study for Validation (DDD) ---
    print("\n" + "="*80)
    print(f">>> VALIDATION: Event Study on DDD Model (LowWage) ({year}) <<<")
    print("="*80)

    formula_es_parts = [
        "found_job ~ C(TreatState) * C(MONTH, Treatment(reference=6)) * LowWage",
        "C(STATEFIP)"
    ]
    if 'log_monthly_cases' in current_control_vars:
        formula_es_parts.insert(1, "log_monthly_cases")
    if 'avg_stringency' in current_control_vars:
        formula_es_parts.insert(1, "avg_stringency")

    final_formula_es = " + ".join(formula_es_parts)
    
    es_data = final_panel_with_controls.copy()
    
    model_es = smf.wls(formula=final_formula_es, data=es_data, weights=es_data['LNKFW1MWT'])
    results_es = model_es.fit(cov_type='cluster', cov_kwds={'groups': es_data['STATEFIP']})
    
    plot_df_ddd = print_did_event_study_coeffs(
        results_es, 
        f"DDD_State_AllAges_HolzerStates_LowWage_{year}", 
        variable="MONTH", 
        ddd_term="LowWage",
        policy_month=policy_month
    )
    plot_did_event_study(plot_df_ddd, f"DDD State Event Study ({year}, LowWage vs Other-Wage)")

In [96]:
# =============================================================================
# CELL 4: MAIN EXECUTION SCRIPT
# =============================================================================

def run_state_did_with_staggered_states(cps_path, policy_path, covid_path, oxcgrt_path):
    """
    Main wrapper function, simplified to use only log_monthly_cases.
    """
    
    # --- Step 0: Define Treatment/Control based on Policy File ---
    policy_df = pd.read_csv(policy_path)
    policy_df['date'] = pd.to_datetime(policy_df['date'], format='%Y-%m-%d', errors='coerce')
    policy_df.dropna(subset=['date'], inplace=True) 

    base_year = 2021
    federal_expiration_date = pd.to_datetime('2021-09-01')

    early_terminators_df = policy_df[
        policy_df['policy_description'].str.contains("end(s|ed) emergency employment benefits", na=False, case=False, regex=True) &
        (policy_df['date'] < federal_expiration_date)
    ].copy()

    early_terminators_df.sort_values(['statefips', 'date'], inplace=True)
    early_terminators_df.drop_duplicates(subset=['statefips'], keep='first', inplace=True)
    staggered_treat_fips = set(early_terminators_df['statefips'].unique())

    if not staggered_treat_fips:
        print("ERROR: No early terminating states found.")
        return

    df_temp_cps = pd.read_csv(cps_path, usecols=['YEAR', 'STATEFIP'])
    cps_states_2021 = set(df_temp_cps[df_temp_cps['YEAR'] == 2021]['STATEFIP'].unique())
    cps_states_2018 = set(df_temp_cps[df_temp_cps['YEAR'] == 2018]['STATEFIP'].unique())
    del df_temp_cps 

    staggered_control_fips_2021 = cps_states_2021 - staggered_treat_fips
    staggered_control_fips_2018 = cps_states_2018 - staggered_treat_fips

    if not staggered_control_fips_2021:
        print("ERROR: No control states found for 2021.")
        return

    print(f"Defined Staggered Treatment States (n={len(staggered_treat_fips)}): {sorted(list(staggered_treat_fips))}")
    print(f"Defined Staggered Control States (n={len(staggered_control_fips_2021)}) for 2021.")
    print(f"Defined Staggered Control States (n={len(staggered_control_fips_2018)}) for 2018.")

    # --- Step 1 & 2: Load Data & Prepare Controls ---
    df = pd.read_csv(cps_path) 
    covid_df = pd.read_csv(covid_path, na_values='.');
    # We still load ox_df, but we won't use it
    ox_df = pd.read_csv(oxcgrt_path, low_memory=False) 

    available_years = sorted(df['YEAR'].unique())

    # --- Prepare COVID Controls ---
    if 'new_case_count' not in covid_df.columns: raise ValueError("'new_case_count' missing.")
    covid_df['new_cases_numeric'] = pd.to_numeric(covid_df['new_case_count'], errors='coerce').fillna(0)
    monthly_covid = covid_df.groupby(['statefips', 'year', 'month'])['new_cases_numeric'].sum().reset_index()
    monthly_covid.rename(columns={'statefips': 'STATEFIP', 'year': 'YEAR', 'month': 'MONTH', 'new_cases_numeric': 'monthly_cases'}, inplace=True)
    monthly_covid['log_monthly_cases'] = np.log(monthly_covid['monthly_cases'] + 1)
    
    # *** REVERTED: We will not use stringency ***
    monthly_stringency = pd.DataFrame()
    control_vars = ['log_monthly_cases']
    
    print("Controls prepared (log_monthly_cases only).")

    # --- Run Main Analysis for 2021 ---
    if 2021 in available_years:
        run_state_did_analysis(
            df=df,
            year=2021,
            holzer_treat_fips=staggered_treat_fips,
            holzer_control_fips=staggered_control_fips_2021,
            monthly_covid=monthly_covid,
            monthly_stringency=monthly_stringency, # Pass empty DF
            control_vars=control_vars,               # Pass list with only log_monthly_cases
            policy_month=7
        )
    else: print("ERROR: Year 2021 not found in CPS data.")

    # --- Run Placebo Test for 2018 ---
    if 2018 in available_years:
        run_state_did_analysis(
            df=df,
            year=2018,
            holzer_treat_fips=staggered_treat_fips,
            holzer_control_fips=staggered_control_fips_2018,
            monthly_covid=monthly_covid,
            monthly_stringency=monthly_stringency, # Pass empty DF
            control_vars=control_vars,               # Pass list with only log_monthly_cases
            policy_month=7
        )
    else: print("WARNING: Year 2018 not found in CPS data. Skipping 2018 placebo.")

In [ ]:
# =============================================================================
# CELL 5: SCRIPT ENTRY POINT 
# =============================================================================

if __name__ == '__main__':

    DATA_DIR = 'data/raw'

    cps_file_path = os.path.join(DATA_DIR, 'cps_00006.csv')
    policy_file_path = os.path.join(DATA_DIR, 'Policy Milestones - State.csv')
    covid_file_path = os.path.join(DATA_DIR, 'COVID - State - Daily.csv')
    oxcgrt_file_path = os.path.join(DATA_DIR, 'OxCGRT_US_latest.csv')
    
    run_state_did_with_staggered_states(
        cps_path=cps_file_path,
        policy_path=policy_file_path,
        covid_path=covid_file_path,
        oxcgrt_path=oxcgrt_file_path
    )

Defined Staggered Treatment States (n=24): [np.int64(1), np.int64(2), np.int64(4), np.int64(5), np.int64(12), np.int64(13), np.int64(16), np.int64(18), np.int64(19), np.int64(22), np.int64(24), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(38), np.int64(39), np.int64(40), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(54), np.int64(56)]
Defined Staggered Control States (n=27) for 2021.
Defined Staggered Control States (n=27) for 2018.
Controls prepared (log_monthly_cases only).

--- RUNNING MAIN ANALYSIS (LowWage) FOR YEAR: 2021 ---
Base 2021 panel (Holzer States, Feb-Aug, valid industry): 12620 obs
  Other Wage (n=8917), Low Wage (n=3703)
Final 2021 panel with controls: 12620 obs

--- DDD Results (2021, All Ages, Holzer States, LowWage) ---
Key Coefficient (TreatState:Post:LowWage): -0.0804
P-value: 0.0351

--- Coefficient Summary (Key Variables) ---
TreatState:Post:LowWage  : Coef =  -0.0804, P-val = 0.0351
TreatState:Post          : Coef =   0.04